In [72]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [12]:
len(embeddings.embed_query("This is a test query to generate embeddings."))

384

In [13]:
from pinecone import Pinecone

In [16]:
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY"))

In [ ]:
from pinecone import ServerlessSpec
# Serverless: Server will be manged by the cloud provider

In [18]:
index_name = "agenticai"

In [23]:
pc.has_index(index_name)

True

In [ ]:
# Creating the index
if not pc.has_index(index_name):
    pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ))

In [27]:
# Loading the index
index = pc.Index(index_name)

In [28]:
from langchain_pinecone import PineconeVectorStore

In [29]:
VectorStore = PineconeVectorStore(index=index, embedding=embeddings)

In [33]:
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)



In [34]:
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [35]:
documents

[Document(metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
 Document(metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic application

In [40]:
#universal identification number
uuids = [str(uuid4()) for _ in range(len(documents))]

In [41]:
uuids

['f8d7e8a8-1d4a-4188-9156-98941c3ee851',
 '1f076382-ee11-48ea-ab79-dca3cbcb1cd6',
 '20815437-afcc-4cea-8f8e-b9d7738424be',
 '1ccc6e60-fcaa-4ff6-a5cb-3e5a56a54b4d',
 'f57beabc-5373-49bf-903b-1afd7de4c73f',
 'f738a98d-f9bf-4185-86a7-9b1f0c9cb8cc',
 '6c568587-d524-4651-b7a2-870fb1b5952f',
 'ce29d51c-36b7-43d5-9a78-5d2a16584eae',
 '6533d11c-c01b-4205-bab8-c46aaef9e059',
 '73cc5054-2ceb-4c9d-84c0-7999620242c0']

In [42]:
VectorStore.add_documents(documents=documents, ids=uuids)

['f8d7e8a8-1d4a-4188-9156-98941c3ee851',
 '1f076382-ee11-48ea-ab79-dca3cbcb1cd6',
 '20815437-afcc-4cea-8f8e-b9d7738424be',
 '1ccc6e60-fcaa-4ff6-a5cb-3e5a56a54b4d',
 'f57beabc-5373-49bf-903b-1afd7de4c73f',
 'f738a98d-f9bf-4185-86a7-9b1f0c9cb8cc',
 '6c568587-d524-4651-b7a2-870fb1b5952f',
 'ce29d51c-36b7-43d5-9a78-5d2a16584eae',
 '6533d11c-c01b-4205-bab8-c46aaef9e059',
 '73cc5054-2ceb-4c9d-84c0-7999620242c0']

In [45]:
results = VectorStore.similarity_search("What lanchain provides to us?", k=2)

In [46]:
results

[Document(id='20815437-afcc-4cea-8f8e-b9d7738424be', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='ce29d51c-36b7-43d5-9a78-5d2a16584eae', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [47]:
results = VectorStore.similarity_search("What lanchain provides to us?", filter={"source": "tweet"})

In [48]:
results

[Document(id='20815437-afcc-4cea-8f8e-b9d7738424be', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='ce29d51c-36b7-43d5-9a78-5d2a16584eae', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='f8d7e8a8-1d4a-4188-9156-98941c3ee851', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(id='f57beabc-5373-49bf-903b-1afd7de4c73f', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again.")]

In [56]:
retriever=VectorStore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3,"score_threshold": 0.5})

In [ ]:
retriever.invoke("What lanchain provides to us?")

[Document(id='73cc5054-2ceb-4c9d-84c0-7999620242c0', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='f8d7e8a8-1d4a-4188-9156-98941c3ee851', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(id='6c568587-d524-4651-b7a2-870fb1b5952f', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [65]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [75]:
from langchain_openai import ChatOpenAI

model=ChatOpenAI(model="o1-mini")

In [67]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [68]:
import pprint
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [78]:
from langchain_core.prompts import PromptTemplate

In [80]:
prompt=PromptTemplate(
    template="expert searcher.\n {context}\n\n{question}",
    input_variables=["context", "question"]
)

In [81]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [82]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [83]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [89]:
print(rag_chain.first)

steps__={'context': VectorStoreRetriever(tags=['PineconeVectorStore', 'HuggingFaceEmbeddings'], vectorstore=<langchain_pinecone.vectorstores.PineconeVectorStore object at 0x0000015FB0464EC0>, search_type='similarity_score_threshold', search_kwargs={'k': 3, 'score_threshold': 0.5})
| RunnableLambda(format_docs), 'question': RunnablePassthrough()}


In [ ]:
rag_chain.invoke("who is Krish?")